In [ ]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import itertools
import plotly.graph_objects as go

# Loading dataset and defining indicators
data = pd.read_csv('../data/fragility_scores.csv')
flip_cols = ['reserves', 'current_account', 'fdi']
data[flip_cols] *= -1
indicators = ['debt', 'inflation', 'reserves', 'fdi', 'current_account']

# Defining our robust minmax function
def robust_minmax(x):
    q10, q90 = x.quantile(0.1), x.quantile(0.9)
    return ((x - q10) / (q90 - q10)).clip(0, 1)

# Grouping data by region and year
region_year = data.groupby(['year', 'region'])['normalised_fragility_score'].mean().reset_index()

# Reshape data from wide format to long format and convert values to integers
data_long = data.melt(id_vars = ['country', 'year'], value_vars = indicators, var_name = 'indicator', value_name = 'value')
data_long['value'] = pd.to_numeric(data_long['value'], errors='coerce')

# Normalise all values using our robust minmax function
data_long['norm'] = data_long.groupby(['indicator'])['value'].transform(robust_minmax)

# Sorting years and countries for our grid
years = sorted(data_long['year'].unique())
countries = sorted(data_long['country'].unique())

# Building a full country-year-indicator grid
grid = pd.DataFrame(itertools.product(
        countries,
        years,
        data_long['indicator'].unique()),columns = ['country', 'year', 'indicator'])

# Merging normalised data from data_long into our grid
data_full = (grid.merge(data_long[['country', 'year', 'indicator', 'norm']], on = ['country', 'year', 'indicator'], how = 'left'))

# Fill missing values by forward-fill, backward-fill, then 0 if needed
data_full['norm'] = data_full.groupby(['country', 'indicator'])['norm'].transform(lambda x: x.fillna(0))

# Defining all_years for our radar plot
all_years = sorted(data_full['year'].unique())

# Normalising fragility scores using robust minmax
data['fragility_score_robust'] = robust_minmax(data['fragility_score'])

In [ ]:
# Creating animated bar chart of average fragility by region
fig = px.bar(
    region_year,
    x = 'region',
    y = 'normalised_fragility_score',
    animation_frame = 'year',
    animation_group = 'region',
    range_y = [0, 1],
    title = 'Average Fragility Score by Region (2010–2021)',
    labels = {'region': 'Region', 'normalised_fragility_score': 'Avg Fragility Score'})

# Adjusting layout for better readability
fig.update_layout(xaxis_tickangle = -39)
fig.show()

In [ ]:
# Defining parameters for the radar plot
fig = px.line_polar(data_full,
    r = 'norm',
    theta = 'indicator',
    color = 'country',
    line_close = True,
    animation_frame = 'year',
    animation_group = 'country',
    category_orders = {'year': all_years},
    title = 'Fragility Radar Profiles (2010–2021)')

# Improving formatting and readability
fig.update_traces(fill='toself', visible='legendonly')
fig.update_layout(polar=dict(radialaxis=dict(range=[0,1], tickformat=".0%")),legend_title_text='Country')

# Displaying the radar plot
fig

In [ ]:
# Finding unique countries and defining modes
countries = sorted(data['country'].unique())
mode_names = ['Standard Min–Max', 'Robust Min–Max']

# Creating a Plotly figure with two blank traces for each mode
fig = go.Figure()
fig.add_trace(go.Scatter(name = mode_names[0], mode = 'lines+markers'))
fig.add_trace(go.Scatter(name = mode_names[1], mode = 'lines+markers'))

# Iterate over countries and create update buttons for the Plotly dropdown
country_buttons = []
for country in countries:
    dataframe = data[data['country'] == country].sort_values('year')
    country_buttons.append(dict(label = country, method = 'update', args = [{
        'x': [dataframe['year'], dataframe['year']], 
        'y': [dataframe['normalised_fragility_score'], dataframe['fragility_score_robust']]}, {'title': f"{country} — {mode_names[0]}"}]))

# Create buttons to toggle visibility between Standard and Robust fragility scores
mode_buttons = [
    dict(label = mode_names[0],
         method = 'update',
         args = [{'visible': [True, False]},
               {'title': f"{countries[0]} — {mode_names[0]}"}]),
    dict(label = mode_names[1],
         method = 'update',
         args = [{'visible': [False, True]},
               {'title': f"{countries[0]} — {mode_names[1]}"}])]

# Loading first country in dataset and set default view to standard min-max
initial_country_data = data[data['country'] == countries[0]].sort_values('year')
fig.data[0].x = initial_country_data['year']
fig.data[0].y = initial_country_data['normalised_fragility_score']
fig.data[1].x = initial_country_data['year']
fig.data[1].y = initial_country_data['fragility_score_robust']
fig.data[0].visible = True
fig.data[1].visible = False

# Updating layout
fig.update_layout(
    margin = dict(t = 120, b = 60, l = 60, r = 20),
    height = 500,
    width = 800,
    updatemenus = [dict(buttons = country_buttons, x = 0.2, y = 1.12, xanchor = "left"),
                   dict(buttons = mode_buttons, x = 0.50, y = 1.12, xanchor = "left")],
    title = f"{countries[0]} — {mode_names[0]}",
    xaxis = dict(tickmode = 'linear', dtick = 1, title = 'Year'),
    yaxis = dict(range = [0,1], title = 'Fragility Score'))

fig.show()

In [ ]:
# Reshape data from wide format to long format
df_long = data.melt(id_vars = ['country', 'year'], value_vars = indicators, var_name = 'indicator', value_name='value')

# Applying robust minmax function to our data
df_long['norm'] = df_long.groupby(['indicator'])['value'].transform(robust_minmax)

# Extracting Zambia's data for 2020
zambia_norm = df_long[(df_long['country'] == 'Zambia') & (df_long['year'] == 2020)]

# Defining parameters for the radar plot
fig = px.line_polar(zambia_norm,
                    r = 'norm',
                    theta = 'indicator',
                    line_close = True,
                    title='Zambia Fragility Profile (2020)')

# Improving formatting and readability
fig.update_traces(fill = 'toself')
fig.update_layout(polar = dict(radialaxis = dict(range = [0, 1], tickformat=".0%")), showlegend = False)

fig.show()

In [ ]:
zambia = data[data['country'] == 'Zambia'].sort_values('year')

# Defining the plot parameters
plt.figure(figsize = (15, 7))
plt.plot(
    zambia['year'],
    zambia['fragility_score_robust'],
    marker = 'o',
    color = 'crimson',
    label = 'Robust Min–Max Fragility')

# Setting plot title, labels, and improving readability
plt.title('Zambia Fragility Score Over Time (2010–2020)')
plt.xlabel('Year')
plt.ylabel('Normalized Fragility (0–1)')
plt.ylim(0, 1)
plt.xticks(zambia['year'])
plt.legend()
plt.grid(True)

plt.show()